# Chess Engine with TensorFlow

## Dataset

In [3]:
import os

files = [file for file in os.listdir("pgn") if file.endswith(".pgn")]

In [5]:
len(files)

1

In [7]:
from chess import pgn

def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

In [9]:
from tqdm import tqdm

LIMIT_OF_FILES = min(len(files), 24)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"pgn/{file}"))
    if (i >= LIMIT_OF_FILES):
        break
    i += 1

  0%|          | 0/1 [00:10<?, ?it/s]


In [36]:
len(games)

9091

## Build & train a neural network

In [39]:
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam # type: ignore
import time

In [41]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix


def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X, y


def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [43]:
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)

In [2]:
import pygame
import sys
import chess
import chess.pgn

# === CONFIG ===
WIDTH, HEIGHT = 640, 640
SQ_SIZE = WIDTH // 8

# === LOAD IMAGES ===
IMAGES = {}
def load_images():
    pieces = ["b_p", "b_r", "b_n", "b_b", "b_q", "b_k",
              "w_p", "w_r", "w_n", "w_b", "w_q", "w_k"]
    for piece in pieces:
        IMAGES[piece] = pygame.transform.scale(
            pygame.image.load("pieces/" + piece + ".png"), (SQ_SIZE, SQ_SIZE)
        )


# === DRAW BOARD ===
def draw_board(screen):
    colors = [pygame.Color("white"), pygame.Color("gray")]
    for row in range(8):
        for col in range(8):
            color = colors[(row + col) % 2]
            pygame.draw.rect(screen, color,
                             pygame.Rect(col*SQ_SIZE, row*SQ_SIZE, SQ_SIZE, SQ_SIZE))

def draw_pieces(screen, board):
    for row in range(8):
        for col in range(8):
            piece = board.piece_at(row*8 + col)
            if piece:
                key = f"{'w' if piece.color else 'b'}_{piece.symbol().lower()}"
                screen.blit(IMAGES[key],
                            pygame.Rect(col*SQ_SIZE, row*SQ_SIZE, SQ_SIZE, SQ_SIZE))

def get_square_under_mouse(pos):
    x, y = pos
    return (y // SQ_SIZE) * 8 + (x // SQ_SIZE)

# === YOUR AI FUNCTION ===
def predict_next_move(board):
    # TODO: Replace with your trained model logic
    # For now, pick a random legal move
    import random
    return random.choice(list(board.legal_moves))

# === MAIN LOOP ===
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Chess GUI with AI")

    load_images()
    board = chess.Board()
    clock = pygame.time.Clock()
    selected_square = None

    # PGN recording
    game = chess.pgn.Game()
    node = game

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                sys.exit()

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_s:  # Save PGN anytime with "S"
                    with open("game_output.pgn", "w") as f:
                        print(game, file=f)
                    print("PGN saved as game_output.pgn")

            elif event.type == pygame.MOUSEBUTTONDOWN:
                clicked_square = get_square_under_mouse(pygame.mouse.get_pos())

                if selected_square is None:
                    piece = board.piece_at(clicked_square)
                    if piece is not None and piece.color == board.turn:
                        selected_square = clicked_square
                else:
                    move = chess.Move(selected_square, clicked_square)
                    if move in board.legal_moves:
                        # Human move
                        board.push(move)
                        node = node.add_variation(move)
                        selected_square = None

                        # AI move
                        if not board.is_game_over():
                            ai_move = predict_next_move(board)
                            if ai_move in board.legal_moves:
                                board.push(ai_move)
                                node = node.add_variation(ai_move)
                    else:
                        selected_square = None

        # Check game over
        if board.is_game_over():
            print("Game over:", board.result())
            with open("game_output.pgn", "w") as f:
                print(game, file=f)
            print("Final PGN saved as game_output.pgn")
            running = False

        draw_board(screen)
        draw_pieces(screen, board)
        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-08-26 22:15:15.331 python[8800:192284] +[IMKClient subclass]: chose IMKClient_Modern
2025-08-26 22:15:15.331 python[8800:192284] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Game over: 1-0
Final PGN saved as game_output.pgn
